# Arab-Andalusian Corpus - Nawba Recognition using Templates from Scores

This notebook computes several experiments, in order to evaluate the performance of nawba recognition algorithms based on templates derived from scores. Each template is synthesized, using Gaussian distributions, from several folded pitch class distributions belonging to a nawba. The folded pitch distribution of a track is compared to the templates and the best match predicts the nawba. 
The experiments test different distance measures and standard deviation values.

## Initialization (MANDATORY)
In this cell, all the libraries are loaded. 
Furthermore, a function checks if the metadata related to Arab-Andalusian corpus of Dunya has been downloaded: if not, all metadata will be downloaded. 
At the end, the code creates an object to manage the Dunya metadata.

#### NB: Before to run, remember to add the Dunya token in the costants.py file. This file is in the directory "utilities".

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.ioff()
import numpy as np

from shutil import copyfile
from utilities.recordingcomputation import *
from utilities.dunyautilities import *
from utilities.metadataStatistics import *
from utilities.generalutilities import *
from utilities.experiments import *

# download metadata from Dunya
if not check_dunya_metadata():
    print("Downloading metadata from Dunya...")
    collect_metadata()

# create an object with all the well-structured metadata
print("Analyzing Dunya Metadata...")
cm = CollectionMetadata()
print("Collection of metadata created")

## Dataset creation (MANDATORY)

An empty object to manage the dataset of the experiments is created.
Then, a list of recordings is imported from a CSV and added to the dataset.

In [ ]:
# create an empty object
do = DataSet(cm)
csv_filename = "dataset_nawba_77_recordings.csv" #"dataset_test.csv" "dataset_nawba_77_recordings.csv"
# add recording mbids of an external file in the dataset 
do.import_dataset_from_csv(csv_filename)

## Nawba Recognition

The parameters of the experiments are defined. An object is created for every experiment and added to a list.
The distance measure parameters are: "city block (L1)", "euclidean (L2)", "correlation", "canberra".
Standard deviation values tested are 20, 30 and 40, but they could be changed.

In [ ]:
distance_measures_list = ["euclidean (L2)"] #["city block (L1)", "euclidean (L2)", "correlation", "canberra"]
random_state = 20
std_list = [30] #[20,30,40]
esperiment_name = "nr_L2_20_30_with_correct_nawba"
source_dir = os.path.join(EXPERIMENT_DIR, esperiment_name)
sub_esperiment_suffix = "exp"

experiment_list = list()
for i_element in range(7):  
    sub_esperiment_name = "{}_{}".format(sub_esperiment_suffix, i_element + 1)
    experiment_list.append(Nawba_Recognition_Experiment(do, i_element, random_state, std_list, distance_measures_list, sub_esperiment_name, source_dir))

In this cell, the experiments will be computed. If the plot_flag is True, the plots of templates and of best matches will be stored in the experiment directory as PNGs.

In [ ]:
counter = 1
plot_flag = True
zip_path = "dataset_nawba_77_recordings.zip"

# check if all the necessary files related to each recording of the dataset are available. 
recordings_with_missing_files = experiment_list[0].get_recordings_without_experiment_files()
# If not import the file from zip
if len(recordings_with_missing_files) != 0:
    extract_files_from_zip(RECORDINGS_DIR, zip_path)
    # second check after unzip
    recordings_with_missing_files = experiment_list[0].get_recordings_without_experiment_files()
    if len(recordings_with_missing_files) != 0:
        raise Exception ("A/some file/s is/are missing")

# run the experiment
for index in range(len(experiment_list)):
    name = "exp_{} results: ".format(counter)
    print()
    print(name)
    experiment_list[index].run(plot_flag=plot_flag)
    experiment_list[index].compute_summary()
    print()
    print(experiment_list[index].df_summary )
    counter += 1

The overall results will be computed and exported in a CSV stored the experiment directory. The confusion matrix of the best parameters combination will be plotted in a PNG file.  

In [ ]:
# compute and print overall results
df_overall =  experiment_list[0].df_summary
for index in range(len(experiment_list)-1):
    df_overall = df_overall.add(experiment_list[index+1].df_summary)
df_overall = df_overall.divide(len(experiment_list))
print(df_overall)

# export the results
export_overall_experiment(experiment_list)

## Test